In [30]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_validate,StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, balanced_accuracy_score, roc_auc_score, make_scorer

## DLL

In [2]:
dll = pd.read_csv('DLLs_Imported.csv')
dll = dll.iloc[:,1:]

In [3]:
X = dll.iloc[:,2:]
y = dll['Type']

In [4]:
X.shape

(29498, 628)

#### naivebayes

In [19]:
from sklearn.naive_bayes import MultinomialNB

# StratifiedKFold 객체 생성
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 교차 검증 및 평가
nb_scores = []

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    

    # Naive Bayes 모델 생성 및 학습
    nb = MultinomialNB()
    nb.fit(X_train, y_train)
    
    # 테스트 데이터 예측 및 평가

    nb_pred = nb.predict(X_test)
    nb_score = accuracy_score(y_test, nb_pred)
    nb_scores.append(nb_score)

# 분류 보고서 출력
print("Naive Bayes Classification Report:")
print(classification_report(y_test, nb_pred))

# 교차 검증 결과 출력
print(f"Naive Bayes: {nb_scores}")
print(f"\nMean accuracy:")
print(f"Naive Bayes: {sum(nb_scores) / len(nb_scores)}")

Naive Bayes Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.29      0.45       375
           1       0.76      0.64      0.70      1004
           2       0.78      0.84      0.81       929
           3       0.76      0.38      0.51       991
           4       0.59      0.63      0.61      1016
           5       0.95      0.09      0.16       844
           6       0.30      0.94      0.46       740

    accuracy                           0.56      5899
   macro avg       0.73      0.54      0.53      5899
weighted avg       0.72      0.56      0.54      5899

Naive Bayes: [0.5584745762711865, 0.5664406779661016, 0.5644067796610169, 0.5656890998474318, 0.5626377352093576]

Mean accuracy:
Naive Bayes: 0.5635297737910189


#### SVM

In [18]:
# StratifiedKFold 객체 생성
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 교차 검증 및 평가
scores = []
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # SVM 모델 생성 및 학습
    svm = SVC(kernel='linear', probability=True)
    svm.fit(X_train, y_train)
    
    # 테스트 데이터 예측 및 평가
    y_pred = svm.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    scores.append(score)
    
    # 분류 보고서 출력
    

# 교차 검증 결과 출력
print(f"Stratified k-fold cross-validation scores: {scores}")
print(f"Mean accuracy: {sum(scores) / len(scores)}")

              precision    recall  f1-score   support

           0       0.88      0.32      0.47       376
           1       0.86      0.65      0.74      1004
           2       0.92      0.89      0.91       929
           3       0.90      0.38      0.54       991
           4       0.61      0.75      0.68      1015
           5       0.31      0.89      0.46       845
           6       0.00      0.00      0.00       740

    accuracy                           0.59      5900
   macro avg       0.64      0.56      0.54      5900
weighted avg       0.65      0.59      0.57      5900



c:\Users\ben81\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ben81\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ben81\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.97      0.37      0.53       376
           1       0.87      0.68      0.76      1004
           2       0.91      0.91      0.91       929
           3       0.92      0.37      0.53       991
           4       0.64      0.79      0.71      1015
           5       0.32      0.92      0.48       845
           6       0.00      0.00      0.00       740

    accuracy                           0.61      5900
   macro avg       0.66      0.58      0.56      5900
weighted avg       0.67      0.61      0.59      5900

              precision    recall  f1-score   support

           0       0.89      0.30      0.45       375
           1       0.88      0.65      0.75      1005
           2       0.93      0.89      0.91       928
           3       0.91      0.38      0.54       992
           4       0.64      0.79      0.71      1015
           5       0.31      0.88      0.46       845
           6       0.37 

#### randomforest

In [7]:
from sklearn.ensemble import RandomForestClassifier
# StratifiedKFold 객체 생성
skf = StratifiedKFold(n_splits=7, shuffle=True, random_state=42)

# 교차 검증 및 평가
scores = []
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    rf = RandomForestClassifier()

# Fit the model to the training data
    rf.fit(X_train, y_train)

# Predict on the test data
    y_pred = rf.predict(X_test)
    print(classification_report(y_test, y_pred))
# Evaluate the model
    score = accuracy_score(y_test, y_pred)
    scores.append(score)
    
print(f"Accuracy: {max(scores)}")


              precision    recall  f1-score   support

           0       0.92      0.35      0.51       269
           1       0.88      0.70      0.78       717
           2       0.94      0.91      0.93       663
           3       0.90      0.42      0.57       708
           4       0.66      0.75      0.70       726
           5       1.00      0.12      0.21       603
           6       0.31      0.97      0.46       528

    accuracy                           0.62      4214
   macro avg       0.80      0.60      0.60      4214
weighted avg       0.80      0.62      0.62      4214

              precision    recall  f1-score   support

           0       0.95      0.36      0.52       268
           1       0.86      0.69      0.76       718
           2       0.94      0.91      0.92       663
           3       0.91      0.39      0.55       708
           4       0.65      0.73      0.69       725
           5       0.99      0.12      0.22       604
           6       0.31 

#### XGBoost

In [5]:
xgb_model = xgb.XGBClassifier(
    objective='multi:softprob',
    tree_method='gpu_hist',
    sampling_method='gradient_based',
    device="gpu"
    )

In [7]:
params = {
    'max_depth':[3,5,7,9],
    'max_leaves':[32, 64, 128, 256],
    'learning_rate':[0.01, 0.05, 0.1, 0.3]    
}
gs = GridSearchCV(xgb_model, params, n_jobs=-1)

gs.fit(X_train, y_train)

c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [23:35:29] WARNING: D:\bld\xgboost-split_1713397764293\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device='gpu',
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_...
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     objective='multi:softprob', ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1, 0.3],
                         'max_depth': [3, 5, 7, 9],
                         'max_leaves': [32, 64, 128, 256]})

In [8]:
dt = gs.best_estimator_
print(dt.score(X_train, y_train))

c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [23:40:21] WARNING: D:\bld\xgboost-split_1713397764293\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [23:40:21] WARNING: D:\bld\xgboost-split_1713397764293\work\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


0.6281697780590336


In [9]:
print(gs.best_params_)

{'learning_rate': 0.3, 'max_depth': 9, 'max_leaves': 64}


In [12]:
y_pred = gs.predict(X_test)

In [13]:
accuracy = gs.score(X_test,y_test)
accuracy

0.6193898305084746

#### Ensemble (SVM,XGB)

In [26]:
from sklearn.ensemble import VotingClassifier

# SVM, XGBoost 모델 객체 생성
svm = SVC()

# Voting 앙상블
ensemble = VotingClassifier(estimators=[('svm', svm), ('xgb', xgb_model)])

# 데이터로 앙상블 모델 학습
ensemble.fit(X_train, y_train)

# 예측
y_pred = ensemble.predict(X_test)

c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [10:46:20] WARNING: D:\bld\xgboost-split_1713397764293\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [10:46:55] WARNING: D:\bld\xgboost-split_1713397764293\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [27]:
ensemble.score(X_test,y_test)

0.6168842176640108

### huristic trial

In [6]:
xgb_model = xgb.XGBClassifier(
    objective='multi:softprob',
    tree_method='gpu_hist',
    sampling_method='gradient_based',
    device="gpu",
    learning_rate= 0.3, 
    max_depth= 9, 
    max_leaves= 64
    )

In [34]:
num_drop = [10,50,100,150,200,300,400,500]
col_idx = np.arange(X.shape[1]) #628
experiment_log = []


for i in num_drop:
    best_performance = 0  # 최고 성능 초기화
    best_score = 0  # 최고 성능을 갖는 실험 초기화
    best_y_pred = None
    best_y_test = None
    for _ in range(10):
        drop_col_idx = np.random.choice(col_idx, size=i, replace=False)
        X_tuned = X.drop(X.columns[drop_col_idx], axis = 1)
        X_train,X_test,y_train,y_test = train_test_split(X_tuned, y, stratify=y ,random_state=42, shuffle=True)
        xgb_model.fit(X_train, y_train)

        splitter = StratifiedKFold(shuffle=True, random_state=42)
        performance_arr = cross_validate(xgb_model,X_train,y_train, cv=splitter)
        performance = np.mean(performance_arr['test_score'])
        score = xgb_model.score(X_test,y_test)
        experiment_log.append([i,performance,score,drop_col_idx])
        if score > best_score and performance > best_performance:
            best_score = score
            best_performance = performance
            y_pred = xgb_model.predict(X_test)
            best_y_pred  = y_pred
            best_y_test = y_test
    else:
        print(f"{i}th completed.")
        print("Best result")
        print("="*100)
        print("Accuracy Score:", accuracy_score(best_y_test, best_y_pred))
        print(confusion_matrix(best_y_test,best_y_pred))



c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [14:37:01] WARNING: D:\bld\xgboost-split_1713397764293\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [14:37:07] WARNING: D:\bld\xgboost-split_1713397764293\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [14:37:11] WARNING: D:\bld\xgboost-split_1713397764293\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` paramet

10th completed.
Best result
Accuracy Score: 0.6196610169491525
[[ 167    2   12    2   29    1  256]
 [   1  867   25   16  333    0   14]
 [   1   48 1057    3   49    2    1]
 [   3   22    8  507   44    0  655]
 [   1   39   39   19  946    0  225]
 [   0    3    2    3   61  122  865]
 [   0   15    0    3    3    0  904]]


c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [14:42:10] WARNING: D:\bld\xgboost-split_1713397764293\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [14:42:15] WARNING: D:\bld\xgboost-split_1713397764293\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [14:42:20] WARNING: D:\bld\xgboost-split_1713397764293\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` paramet

50th completed.
Best result
Accuracy Score: 0.6200677966101695
[[ 168    1   11    2   30    1  256]
 [   1  866   25   17  333    0   14]
 [   1   47 1057    3   49    2    2]
 [   2   20    9  510   43    0  655]
 [   1   39   39   19  946    0  225]
 [   2    3    2    1   61  122  865]
 [   0   15    0    3    3    0  904]]


c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [14:47:10] WARNING: D:\bld\xgboost-split_1713397764293\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [14:47:15] WARNING: D:\bld\xgboost-split_1713397764293\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [14:47:20] WARNING: D:\bld\xgboost-split_1713397764293\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` paramet

...

c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [15:02:26] WARNING: D:\bld\xgboost-split_1713397764293\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [15:02:29] WARNING: D:\bld\xgboost-split_1713397764293\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [15:02:32] WARNING: D:\bld\xgboost-split_1713397764293\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` paramet

400th completed.
Best result
Accuracy Score: 0.5640677966101695
[[113  12  13   5  69   0 257]
 [  0 836  67  14 325   0  14]
 [  2 179 884  12  83   1   0]
 [  2  25  28 454  82   1 647]
 [  1  28  84  16 910   0 230]
 [  0   6   3   6  85  94 862]
 [  0  51   1   3   1   0 869]]


c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [15:05:41] WARNING: D:\bld\xgboost-split_1713397764293\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [15:05:44] WARNING: D:\bld\xgboost-split_1713397764293\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [15:05:47] WARNING: D:\bld\xgboost-split_1713397764293\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` paramet

500th completed.
Best result
Accuracy Score: 0.5037288135593221
[[ 57   4  31   1 119   1 256]
 [  0 627 298   2 315   0  14]
 [  1  79 917   0 164   0   0]
 [  2   4 183 298  89  16 647]
 [  1   1 187   0 854   1 225]
 [  0  29   3   2  64  94 864]
 [  0   0  54   1   2   0 868]]


c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [15:08:14] WARNING: D:\bld\xgboost-split_1713397764293\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [15:08:14] WARNING: D:\bld\xgboost-split_1713397764293\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [36]:
experiment_log

[[10,
  0.619897947507688,
  0.6196610169491525,
  array([517, 456, 405, 368,   8, 609, 520, 504, 296,  82])],
 [10,
  0.6187227653988006,
  0.6192542372881356,
  array([492,  70, 206, 181, 414, 482, 189, 306, 521,  15])],
 [10,
  0.6169598287716717,
  0.6153220338983051,
  array([ 46, 510,  11, 598, 253,  86, 212, 112, 513, 252])],
 [10,
  0.6132081915795711,
  0.6136949152542372,
  array([209,  56, 388,  84, 361, 367, 307,  45, 441, 120])],
 [10,
  0.6195363553702966,
  0.6196610169491525,
  array([493, 559, 535, 614, 363, 268,  24, 157, 280, 408])],
 [10,
  0.6123945096596889,
  0.6165423728813559,
  array([364, 206, 216, 305, 367, 170, 362, 438, 146,   5])],
 [10,
  0.6195364166692208,
  0.6191186440677966,
  array([444, 172, 254, 189, 227, 208, 295, 594, 121, 546])],
 [10,
  0.6197623951532985,
  0.6202033898305085,
  array([ 94, 478, 392, 515, 248, 402,  55, 447, 267,  68])],
 [10,
  0.6195815633268971,
  0.6195254237288136,
  array([225,  15, 308, 590, 449, 400, 509, 317, 321, 3

In [42]:
extracted_data = [[e[0], e[1], e[2]] for e in experiment_log]
# extracted_data = np.arra
experiment_df = pd.DataFrame(data=extracted_data,columns=['i','performance','score'])
# experiment_df.to_excel('experiment_result.xlsx')
experiment_df

,i,performance,score
0,10,0.619898,0.619661
1,10,0.618723,0.619254
2,10,0.616960,0.615322
3,10,0.613208,0.613695
4,10,0.619536,0.619661
...,...,...,...
75,500,0.246666,0.247593
76,500,0.359806,0.362576
77,500,0.215613,0.216949
78,500,0.364959,0.365695


In [43]:
experiment_df.max()

i              500.000000
performance      0.619898
score            0.620203
dtype: float64

## PE_Header

In [15]:
pe_h = pd.read_csv('PE_Header.csv')
pe_h = pe_h.iloc[:,1:]

X = pe_h.iloc[:,2:]
Y = pe_h['Type']
X_train,X_test,y_train,y_test = train_test_split(X, Y, stratify=Y, random_state=42, shuffle=True)


xgb_model = xgb.XGBClassifier(
    objective='multi:softprob',
    tree_method='gpu_hist',
    sampling_method='gradient_based',
    device="gpu"
    )


params = {
    'max_depth':[3,5,7,9],
    'max_leaves':[32, 64, 128, 256],
    'learning_rate':[0.01, 0.05, 0.1, 0.3]    
}
gs_h = GridSearchCV(xgb_model, params, n_jobs=-1)

gs_h.fit(X_train, y_train)

c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [23:59:24] WARNING: D:\bld\xgboost-split_1713397764293\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device='gpu',
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_...
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     objective='multi:softprob', ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1, 0.3],
                         'max_depth': [3, 5, 7, 9],
                         'max_leaves': [32, 64, 128, 256]})

In [16]:
dt = gs_h.best_estimator_
print(dt.score(X_train, y_train))
print(gs_h.best_params_)
print(gs_h.score(X_test,y_test))

0.9767389845672109
{'learning_rate': 0.3, 'max_depth': 9, 'max_leaves': 64}
0.8844605475040258


c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [23:59:27] WARNING: D:\bld\xgboost-split_1713397764293\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


## PE_Section

In [17]:
pe_s = pd.read_csv('PE_Section.csv')
pe_s = pe_s.iloc[:,1:]

X = pe_s.iloc[:,2:]
Y = pe_s['Type']
X_train,X_test,y_train,y_test = train_test_split(X, Y, stratify=Y, random_state=42, shuffle=True)


xgb_model = xgb.XGBClassifier(
    objective='multi:softprob',
    tree_method='gpu_hist',
    sampling_method='gradient_based',
    device="gpu"
    )


params = {
    'max_depth':[3,5,7,9],
    'max_leaves':[32, 64, 128, 256],
    'learning_rate':[0.01, 0.05, 0.1, 0.3]    
}
gs_s = GridSearchCV(xgb_model, params, n_jobs=-1)

gs_s.fit(X_train, y_train)

c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [00:16:47] WARNING: D:\bld\xgboost-split_1713397764293\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device='gpu',
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_...
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     objective='multi:softprob', ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1, 0.3],
                         'max_depth': [3, 5, 7, 9],
                         'max_leaves': [32, 64, 128, 256]})

In [18]:
dt = gs_s.best_estimator_
print(dt.score(X_train, y_train))
print(gs_s.best_params_)
print(gs_s.score(X_test,y_test))

0.9399641577060932
{'learning_rate': 0.3, 'max_depth': 9, 'max_leaves': 128}
0.8161290322580645


c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [00:16:51] WARNING: D:\bld\xgboost-split_1713397764293\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [21]:
import joblib


# 모델 저장
joblib.dump(gs_h, 'gs_h.pkl')
joblib.dump(gs_s, 'gs_s.pkl')
joblib.dump(dll, 'dll.pkl')


['dll.pkl']

In [22]:
model = joblib.load('gs_s.pkl')
model.score(X_test,y_test)

c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [02:01:03] WARNING: D:\bld\xgboost-split_1713397764293\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


0.8161290322580645

## Improved

In [2]:
pe_h = pd.read_csv('PE_Header.csv')
pe_h = pe_h.iloc[:,1:]

X = pe_h.iloc[:,2:]
Y = pe_h['Type']
X_train,X_test,y_train,y_test = train_test_split(X, Y, stratify=Y, random_state=42, shuffle=True)


xgb_model = xgb.XGBClassifier(
    objective='multi:softprob',
    tree_method='gpu_hist',
    sampling_method='gradient_based',
    device="gpu",
    # 최적 파라미터 반영
    learning_rate= 0.3, 
    max_depth= 9, 
    max_leaves= 64
    )

# min_child_weight and gamma.
params = {
    'gamma':[3,5,7,9,15,20]
}
gs_h = GridSearchCV(xgb_model, params, n_jobs=-1)

gs_h.fit(X_train, y_train)

c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [09:19:56] WARNING: D:\bld\xgboost-split_1713397764293\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device='gpu',
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=0.3, max_bin=None,
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=9,
                                     max_leaves=64, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     objective='multi:softprob', ...),
             n_jobs=-1, param_grid={'gamma': [3, 5, 7, 9, 15, 20]})

In [3]:
dt = gs_h.best_estimator_
print(dt.score(X_train, y_train))
print(gs_h.best_params_)
print(gs_h.score(X_test,y_test))

0.8913442182956833
{'gamma': 3}
0.8666129898013956


c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [09:20:01] WARNING: D:\bld\xgboost-split_1713397764293\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [09:20:01] WARNING: D:\bld\xgboost-split_1713397764293\work\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


In [4]:
pe_s = pd.read_csv('PE_Section.csv')
pe_s = pe_s.iloc[:,1:]

X = pe_s.iloc[:,2:]
Y = pe_s['Type']
X_train,X_test,y_train,y_test = train_test_split(X, Y, stratify=Y, random_state=42, shuffle=True)


xgb_model = xgb.XGBClassifier(
    objective='multi:softprob',
    tree_method='gpu_hist',
    sampling_method='gradient_based',
    device="gpu",
    # 최적 파라미터 반영
    learning_rate= 0.3, 
    max_depth= 9, 
    max_leaves= 128
    )


params = {
    'gamma':[3,5,7,9,15,20]
}
gs_s = GridSearchCV(xgb_model, params, n_jobs=-1)

gs_s.fit(X_train, y_train)

c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [09:22:20] WARNING: D:\bld\xgboost-split_1713397764293\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device='gpu',
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=0.3, max_bin=None,
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=9,
                                     max_leaves=128, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     objective='multi:softprob', ...),
             n_jobs=-1, param_grid={'gamma': [3, 5, 7, 9, 15, 20]})

In [5]:
dt = gs_s.best_estimator_
print(dt.score(X_train, y_train))
print(gs_s.best_params_)
print(gs_s.score(X_test,y_test))

0.8261200716845878
{'gamma': 3}
0.7935483870967742


c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [09:23:08] WARNING: D:\bld\xgboost-split_1713397764293\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
